In [3]:

!pip install transformers datasets accelerate evaluate --upgrade


In [4]:
import transformers
print(transformers.__version__)


5.0.0


In [5]:
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)


In [6]:
dataset = load_dataset("dair-ai/emotion")

dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [7]:
model_checkpoint = "distilroberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [8]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length"
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=6
)


Loading weights:   0%|          | 0/101 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: distilroberta-base
Key                         | Status     | 
----------------------------+------------+-
roberta.pooler.dense.bias   | UNEXPECTED | 
roberta.pooler.dense.weight | UNEXPECTED | 
lm_head.layer_norm.weight   | UNEXPECTED | 
lm_head.bias                | UNEXPECTED | 
lm_head.dense.bias          | UNEXPECTED | 
lm_head.dense.weight        | UNEXPECTED | 
lm_head.layer_norm.bias     | UNEXPECTED | 
classifier.dense.bias       | MISSING    | 
classifier.dense.weight     | MISSING    | 
classifier.out_proj.weight  | MISSING    | 
classifier.out_proj.bias    | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [10]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [21]:
training_args = TrainingArguments(
    output_dir="distilroberta-emotion",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=50
)


In [17]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))


In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
   train_dataset=small_train_dataset,
eval_dataset=small_eval_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)



In [23]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.479632,0.382166,0.878000


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=125, training_loss=0.45228084564208987, metrics={'train_runtime': 100.8131, 'train_samples_per_second': 19.839, 'train_steps_per_second': 1.24, 'total_flos': 264953696256000.0, 'train_loss': 0.45228084564208987, 'epoch': 1.0})

In [24]:
trainer.save_model("distilroberta-demo-model")
tokenizer.save_pretrained("distilroberta-demo-model")


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('distilroberta-demo-model/tokenizer_config.json',
 'distilroberta-demo-model/tokenizer.json')

In [25]:
!zip -r distilroberta-demo-model.zip distilroberta-demo-model


  adding: distilroberta-demo-model/ (stored 0%)
  adding: distilroberta-demo-model/tokenizer_config.json (deflated 50%)
  adding: distilroberta-demo-model/model.safetensors (deflated 7%)
  adding: distilroberta-demo-model/config.json (deflated 55%)
  adding: distilroberta-demo-model/training_args.bin (deflated 53%)
  adding: distilroberta-demo-model/tokenizer.json (deflated 82%)


In [26]:
from google.colab import files
files.download("distilroberta-demo-model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>